In [1]:
%load_ext autoreload
%autoreload 2

import builtins
import IPython.lib.deepreload
builtins.reload = IPython.lib.deepreload.reload # type: ignore

In [2]:
import os
from functools import reduce
from operator import add
from typing import List

import numpy as np

import swcgeom
from swcgeom.analysis import draw

In [11]:
def collect_tips(cur: swcgeom.Tree.Node, children: List[int]) -> int:
    cur["tips"] = max(1, reduce(add, children, 0))
    return cur["tips"]

def collect_length(cur: swcgeom.Tree.Node, children: List[swcgeom.Tree.Node]) -> swcgeom.Tree.Node:
    def length(acc: float, child: swcgeom.Tree.Node) -> float:
        return acc + cur.distance(child) + child["length"]

    cur["length"] = reduce(length, children, 0.0)
    return cur

def add_tips_and_length(tree: swcgeom.Tree, dest: str) -> None:
    tree.ndata["tips"] = np.zeros_like(tree.id(), dtype=np.int32)
    total_tips = tree.traverse(leave=collect_tips)

    tree.ndata["length"] = np.zeros_like(tree.id(), dtype=np.float32)
    root_node = tree.traverse(leave=collect_length)
    tree.ndata["r"] = tree.ndata["length"] / root_node["length"] * 100

    tree.to_swc(dest, extra_cols=["tips", "length"])

In [ ]:
swc = "101711-10_4p5-of-16_initial.CNG.swc"
tree = swcgeom.Tree.from_swc(swc)
add_tips_and_length(tree, "test.swc")

In [ ]:
for swc in swcgeom.Population.from_swc("/data/VPM/vpm_axon_with_subtree_length_ratio/"):
    print(swc.source)
    add_tips_and_length(swc, f"./test/{swc.source.split('/')[-1]}")